In [1]:
%load_ext autoreload
%autoreload 2
    
import os, sys

os.environ['DYLD_LIBRARY_PATH']        = os.path.join(os.environ.get('CONDA_PREFIX',''), 'lib')
os.environ['DYLD_FALLBACK_LIBRARY_PATH'] = ''
#PWD = os.getenv('PWD')
PWD = '/Users/chris/Documents/lab/emAnalysis/backend/server'

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

init_django("server")
from django.conf import settings
media_root = settings.MEDIA_ROOT
import numpy as np
import os
from matplotlib import pyplot as plt
import functools
import math
import sims
import copy
import cv2
import os
from pathlib import Path
import numpy as np
from skimage.measure import regionprops
from scipy import spatial
import pickle
from matplotlib.figure import Figure
import pyvips
from core.models import Canvas
from mims.models import MIMSImage, MIMSImageSet
from scipy.ndimage import rotate
import pprint
import sims
import SimpleITK as sitk
import tifffile

from mims.services.image_utils import image_from_im_file
from mims.services.register import validate_mims_image, get_mims_dims, get_points_transform, polygon_centroid, _flip_x



OSError: cannot load library 'libvips.42.dylib': dlopen(libvips.42.dylib, 0x0002): tried: 'libvips.42.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibvips.42.dylib' (no such file), '/Users/chris/miniforge3/envs/bioio/bin/../lib/libvips.42.dylib' (no such file), '/usr/lib/libvips.42.dylib' (no such file, not in dyld cache), 'libvips.42.dylib' (no such file).  Additionally, ctypes.util.find_library() did not manage to locate a library called 'libvips.42.dylib'

In [8]:
# Jupyter cell — stitch every MIMSImage in a Canvas into one big 16-bit PNG per isotope
# ------------------------------------------------------------------------------------
from pathlib import Path
import numpy as np
import cv2
from django.shortcuts import get_object_or_404
from mims.models import Canvas, MIMSImage         # adjust import path if necessary
from skimage.transform import (SimilarityTransform,
                               ThinPlateSplineTransform, warp)
from PIL import Image
from mims.services.image_utils import image_from_im_file
from mims.services.register import validate_mims_image, get_mims_dims, get_points_transform, polygon_centroid, _flip_x

# ───────────────────────── helpers ────────────────────────────────────────────────
ISOTOPES = ["14N 12C", "13C", "12C", "197Au", "32S"]
UINT8_ISO = {"197Au"}
RATIO_PAIR = ("13C", "12C") 

def slug(s: str) -> str:
    return s.replace(" ", "_").replace("/", "_")

def blank_canvas(dtype, w, h):
    return np.zeros((h, w), dtype=dtype)

def save_png(p: Path, arr: np.ndarray):
    p.parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(p), arr, [cv2.IMWRITE_PNG_COMPRESSION, 3])

# ───────────────────── registration geometry per-image ───────────────────────────
def registration_geometry(mims_img):
    """Return dict holding warp info that is isotope-independent."""
    (_, em_shapes, mims_shapes,
     em_pts,   mims_pts)  = validate_mims_image(mims_img.id)
    h_mims, w_mims        = get_mims_dims(mims_img)

    # coarse similarity (+ mirror) ------------------------------------------------
    em_cent   = [polygon_centroid(s) for s in em_shapes]  + em_pts
    mims_cent = [polygon_centroid(s) for s in mims_shapes]+ mims_pts
    base_tf, needs_flip, max_x = get_points_transform(
        mims_img.image_set,
        np.asarray(mims_cent,float),
        np.asarray(em_cent,  float)
    )
    if needs_flip:
        mims_shapes = [_flip_x(s,max_x) for s in mims_shapes]
        mims_pts    = [_flip_x(p,max_x) for p in mims_pts]

    # landmark stacks -------------------------------------------------------------
    mims_stack, em_stack = np.empty((0,2)), np.empty((0,2))
    for mp, ep in zip(mims_shapes, em_shapes):
        mims_stack = np.vstack([mims_stack, polygon_centroid(mp)])
        em_stack   = np.vstack([em_stack,   polygon_centroid(ep)])
    for mp, ep in zip(mims_pts, em_pts):
        mims_stack = np.vstack([mims_stack, mp])
        em_stack   = np.vstack([em_stack,   ep])
    
    if len(mims_stack) < 5 or len(em_stack) < 5:
        raise ValueError(f"Not enough landmarks found for image {mims_img.id}")

    mims_pred  = base_tf(mims_stack)
    offset     = np.median(mims_pred - em_stack, axis=0)

    mims_tf = SimilarityTransform(scale=1,
                                  rotation=base_tf.rotation,
                                  translation=[0,0])
    em_tf   = SimilarityTransform(scale=1/base_tf.scale,
                                  translation=-(base_tf.translation-offset)/base_tf.scale)

    # ensure everything lives in +ve canvas coords -------------------------------
    mims_corners = np.array([[0,0],[w_mims-1,0],[w_mims-1,h_mims-1],[0,h_mims-1]], float)
    if needs_flip:  mims_corners = _flip_x(mims_corners, max_x)
    em_corners     = mims_tf(mims_corners)
    shift          = -np.minimum(np.min(em_corners,0), 0)
    if (shift>0).any():
        mims_tf = SimilarityTransform(scale=1,
                                      rotation=base_tf.rotation,
                                      translation=shift)
        em_tf   = SimilarityTransform(scale=1/base_tf.scale,
                                      translation=-(base_tf.translation-offset)/base_tf.scale + shift)
        em_corners = mims_tf(mims_corners)

    mims_pred = mims_tf(mims_stack);   em_pred = em_tf(em_stack)
    mims_pred = np.vstack([mims_pred])
    em_pred   = np.vstack([em_pred])

    # EM crop box for this image --------------------------------------------------
    canvas_corners = mims_tf(mims_corners)
    em_corners2    = em_tf.inverse(canvas_corners)
    x0,y0 = np.floor(em_corners2.min(0)).astype(int)
    x1,y1 = np.ceil (em_corners2.max(0)).astype(int)
    em_img = np.array(Image.open(mims_img.canvas.images.first().file.path))
    H,W    = em_img.shape[:2]
    precrop_bbox = [x0,y0,x1,y1].copy()
    x0,x1  = np.clip([x0,x1], 0, W);  y0,y1 = np.clip([y0,y1], 0, H)

    out_shape = [int(em_corners[:,1].ptp()), int(em_corners[:,0].ptp())]

    # thin-plate spline -----------------------------------------------------------
    tps = ThinPlateSplineTransform();  tps.estimate(em_pred, mims_pred)

    #print("── TPS internal dict ──", mims_img.id, mims_img.name)
    #for k,v in tps.__dict__.items():
    #    print(f"{k:15s}:", type(v), getattr(v, "shape", v))
    #print("── end dump ──")

    return dict(needs_flip=needs_flip, max_x=max_x,
                mims_tf=mims_tf, tps=tps,
                bbox=(x0,y0,x1,y1),
                precrop_bbox=precrop_bbox,
                out_shape=out_shape)

def print_stats(img_name, img):
    vmin, vmax = img.min(), img.max()
    nonzero = np.count_nonzero(img)
    print(f"{img_name}  min = {vmin}, max = {vmax}, non-zero pixels = {nonzero} of {img.shape[0]*img.shape[1]}\n")
    return
# ─────────────────────────── patch generator ─────────────────────────────────────
def isotope_patch(mims_img, iso_name, geom):
    """
    Returns:
        patch  : (h, w) uint16 greyscale
        x0, y0 : top-left placement on full canvas
    """
    name = mims_img.name
    src = image_from_im_file(mims_img.file.path, iso_name, autocontrast=False)
    if geom["needs_flip"]:
        src = src[:, ::-1]

    img = warp(src, geom["mims_tf"].inverse,
               output_shape=geom["out_shape"], preserve_range=True, order=0)
    img = warp(img, geom["tps"],
               output_shape=geom["out_shape"], preserve_range=True, order=0)

    # build binary mask once to zero background
    ones = np.ones_like(src, np.uint8)
    mask = warp(ones, geom["mims_tf"].inverse,order=0,
                output_shape=geom["out_shape"], cval=0, preserve_range=True)
    mask = warp(mask, geom["tps"],order=0,
                output_shape=geom["out_shape"], cval=0, preserve_range=True)

    x0, y0, x1, y1 = geom["bbox"]
    x0p, y0p, x1p, y1p = geom["precrop_bbox"]
    xcrop_start = max(0, x0 - x0p)
    xcrop_end = (x1 - x1p)
    if xcrop_end >= 0:
        xcrop_end = x1
    ycrop_start = max(0, y0 - y0p)
    ycrop_end = (y1 - y1p)
    if ycrop_end >= 0:
        ycrop_end = y1

    img  = cv2.resize(img,  (x1p - x0p, y1p - y0p), interpolation=cv2.INTER_NEAREST)
    img = img[ycrop_start:ycrop_end, xcrop_start:xcrop_end]
    mask = cv2.resize(mask, (x1p - x0p, y1p - y0p), interpolation=cv2.INTER_NEAREST)
    mask = mask[ycrop_start:ycrop_end, xcrop_start:xcrop_end]

    patch = np.clip(img, 0, 65535).astype(np.uint16)
    patch[mask == 0] = 0                                  # outside source → black
    return patch, x0, y0

# ─────────────────────────── main driver ─────────────────────────────────────────
def stitch_canvas(canvas_name: str, root_out: str):
    canvas  = get_object_or_404(Canvas, name=canvas_name)
    out_dir = Path(root_out) / canvas.name
    out_dir.mkdir(parents=True, exist_ok=True)

    # pre-compute warp geometry once per MIMSImage
    geom_cache = {}
    for m in MIMSImage.objects.filter(canvas=canvas):
        try:
            geom_cache[m.id] = registration_geometry(m)
        except Exception as e:
            print(f"Error registering {m.id}: {e}")
            continue

    # hold these two for the later ratio
    ratio_bufs = {}
    for iso in ISOTOPES:
        use_u8   = iso in UINT8_ISO
        dtype    = np.uint8  if use_u8 else np.uint16
        big      = blank_canvas(dtype, canvas.width, canvas.height)

        for mims_img in MIMSImage.objects.filter(canvas=canvas, id__in=geom_cache.keys()).order_by("-name"):
            if not mims_img.isotopes.filter(name=iso).exists() or mims_img.id not in geom_cache:
                continue
            patch, x0, y0 = isotope_patch(mims_img, iso, geom_cache[mims_img.id])
            if use_u8:
                patch = np.clip(patch, 0, 255).astype(np.uint8)

            h, w   = patch.shape
            region = big[y0:y0+h, x0:x0+w]


            try:
                write_mask = (patch > 0) & (region == 0)          # only paint where dst==0
                region[write_mask] = patch[write_mask]
            except Exception as e:
                print(f"Error writing {iso} to {mims_img.name}: {e}")
                print(f"Dims are {canvas.width}x{canvas.height}")
                print(f"patch: {patch.shape}, region: {region.shape}, x0: {x0}, w: {w}, y0: {y0}, h: {h}")
                raise e
            big[y0:y0+h, x0:x0+w] = region

        out_filename = f"{iso.replace(' ', '_').replace('/', '_')}.png"
        save_png(out_dir / out_filename, big)

        print(f"✓ saved {iso}")

        if iso in RATIO_PAIR:            # keep for later ratio
            ratio_bufs[iso] = big.astype(np.uint32)   # promote to avoid overflow

    # ------------- build the 13C ratio (13C / (13C+12C) * 10 000) --------------
    print("Computing 13C_ratio …")
    c13 = ratio_bufs["13C"]
    c12 = ratio_bufs["12C"]
    denom = c13 + c12

    ratio = np.zeros_like(c13, dtype=np.float32)
    np.divide(c13, denom, out=ratio, where=denom > 0)
    ratio = (ratio * 1.0e4).round()
    ratio_u8 = np.clip(ratio, 0, 255).astype(np.uint8)
    save_png(out_dir / "13C_ratio.png", ratio_u8)
    print(f"✓ saved 13C_ratio → {out_dir/'13C_ratio.png'}")

    # free memory that we kept for ratio
    del ratio_bufs, c13, c12, ratio

    print("All isotopes and ratio complete.")

output_folder = "/Volumes/Chris_SSD/ImmunoEM-registered"
# ─────────────────────────── example call ────────────────────────────────────────
#for canvas in "75_6hrfast_M2",
for canvas in Canvas.objects.all()[0:1]:
    print(f"Starting {canvas}...")
    #stitch_canvas(canvas, output_folder)


Starting 73_6hrfast_M1...


KeyboardInterrupt: 

In [9]:
import os
from pathlib import Path

import numpy as np
from PIL import Image

def analyze_197Au_images(input_folder):
    """
    Recursively find every '197Au.png' under input_folder,
    load it as a NumPy array, and print min, max, and non-zero count.
    """
    input_folder = Path(input_folder)
    for img_path in input_folder.rglob('*197Au.png'):
        if img_path.parts[-1][0] == '.':
            continue
        # Load image
        arr = np.array(Image.open(img_path))
        # Compute stats
        vmin, vmax = arr.min(), arr.max()
        nonzero = np.count_nonzero(arr)
        print(f"{img_path.parts}  min = {vmin}, max = {vmax}, non-zero pixels = {nonzero}\n")
inputf = "/Volumes/Chris_SSD/ImmunoEM-registered/"
#analyze_197Au_images(inputf)

('/', 'Volumes', 'Chris_SSD', 'ImmunoEM-registered', '73_6hrfast_M1', '197Au.png')  min = 0, max = 233, non-zero pixels = 4005135

('/', 'Volumes', 'Chris_SSD', 'ImmunoEM-registered', '75_6hrfast_M2', '197Au.png')  min = 0, max = 19, non-zero pixels = 2560264

('/', 'Volumes', 'Chris_SSD', 'ImmunoEM-registered', '77_6hrfast_M3', '197Au.png')  min = 0, max = 37, non-zero pixels = 3211472

('/', 'Volumes', 'Chris_SSD', 'ImmunoEM-registered', '79_1hr13C_M1', '197Au.png')  min = 0, max = 30, non-zero pixels = 3756959

('/', 'Volumes', 'Chris_SSD', 'ImmunoEM-registered', '89_ONfast_M3', '197Au.png')  min = 0, max = 255, non-zero pixels = 13467197

('/', 'Volumes', 'Chris_SSD', 'ImmunoEM-registered', '81_1hr13C_M2', '197Au.png')  min = 0, max = 255, non-zero pixels = 4831010

('/', 'Volumes', 'Chris_SSD', 'ImmunoEM-registered', '83_1hr_13C_M3', '197Au.png')  min = 0, max = 35, non-zero pixels = 5194335

('/', 'Volumes', 'Chris_SSD', 'ImmunoEM-registered', '85_ONfast_M1', '197Au.png')  min = 

In [ ]:
from pathlib import Path
import shutil

def delete_final_subfolders(root_dir: str):
    """
    Recursively delete all subdirectories named 'final' under root_dir.

    Parameters
    ----------
    root_dir : str
        Path to the top‐level directory in which to search.
    """
    root = Path(root_dir)
    # Find every folder named exactly 'final'
    for final_dir in root.rglob('final'):
        if final_dir.is_dir():
            try:
                #shutil.rmtree(final_dir)
                print(f"Deleted: {final_dir}")
            except Exception as e:
                print(f"Failed to delete {final_dir}: {e}")

# Example usage:
#delete_final_subfolders('/Users/chris/Documents/lab/emAnalysis/backend/server/media')


Deleted: /Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/2149ac1d-9bc2-4c2e-b9dd-603d6e80a1b6/mims_images/final
Deleted: /Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/a4a35f29-9339-40b3-8df6-7fed4c754776/mims_images/final
Deleted: /Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/a85bfc30-9cd5-44f5-aa60-d97619053011/mims_images/final
Deleted: /Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/acc89214-000e-4ae7-b219-ea8948300dc6/mims_images/final
Deleted: /Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/881924a6-1798-409b-8e0e-5405df9de9df/mims_images/final
Deleted: /Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/34de9890-d880-46d9-9ee3-ccd9f2cad94c/mims_images/final
Deleted: /Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/5a2ec271-ff81-40a9-9e62-2fb520ec441c/mims_images/final
Deleted: /Users/chris/Docum

In [2]:
from mims.models import Canvas
from mims.services.register2 import stitch_canvas

# ─────────────────────────── example call ────────────────────────────────────────
# for canvas in "75_6hrfast_M2",
for canvas in Canvas.objects.all()[0:1]:
    print(f"Starting {canvas}...")
    stitch_canvas(canvas)


Starting 73_6hrfast_M1...
Stitching a49c7172-8f55-4960-9752-8b669b20014d - from 2025-03-31 14:16:20.562907+00:00
Error registering 78e37df5-101e-478e-b692-3b7ea8233316: Shape file not found: /Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/a49c7172-8f55-4960-9752-8b669b20014d/mims_images/73_ROI_5/registration/reg_shapes.json
Error registering 7a04498b-4f8b-4b18-a867-cef62f41dcb0: Shape file not found: /Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/a49c7172-8f55-4960-9752-8b669b20014d/mims_images/73_ROI_6/registration/reg_shapes.json
Error registering 826c3568-5b06-46f5-adb3-acb2a1cc20cf: Shape file not found: /Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/a49c7172-8f55-4960-9752-8b669b20014d/mims_images/73_ROI_7/registration/reg_shapes.json
Error registering f29b2bdf-d647-40d7-888a-18d6022d1d1f: Shape file not found: /Users/chris/Documents/lab/emAnalysis/backend/server/media/mims_image_sets/a49c7172-8f55-

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
# cell 1: prerequisites
import os
import xml.etree.ElementTree as ET
import numpy as np
from PIL import Image
import json
import math
import matplotlib.pyplot as plt

# cell 2: point to your .dzi file
info_json_path = '/Users/chris/Documents/lab/emAnalysis/backend/server/media/mims/mosaics/set_881924a6-1798-409b-8e0e-5405df9de9df/13C/mosaic_20250708T230647_13C.tif/info.json'
with open(info_json_path, 'r') as f:
    info = json.load(f)


# extract dims and tile specs
full_width   = info['width']
full_height  = info['height']

# IIIF3 uses a "tiles" array; we take the first entry
tile_spec    = info['tiles'][0]
tile_w       = tile_spec['width']
tile_h       = tile_spec.get('height', tile_w)

# scaleFactors are the downsampling factors relative to full size
levels       = sorted(tile_spec['scaleFactors'])

# pick a level: 1 = full-res, larger = more downsampled
# here we pick the middle level
level        = levels[len(levels)//2]

# figure out how many tiles across/down at this level
cols = math.ceil(full_width  / (tile_w * level))
rows = math.ceil(full_height / (tile_h * level))

# locate tile folder (sits alongside info.json)
tile_folder  = os.path.dirname(info_json_path)

# detect extension from descriptor (fallback to first file if needed)

ext = "png"

# collect and stitch tiles
mosaic = np.zeros((rows*tile_h, cols*tile_w, 3), dtype=np.uint8)
for fname in os.listdir(tile_folder):
    x_start, y_start, x_end, y_end = fname.split(',')

    tile_path = os.path.join(tile_folder, fname)
    img = Image.open(tile_path)
    arr = np.array(img)

    h, w = arr.shape[:2]
    mosaic[row*tile_h: row*tile_h+h,
           col*tile_w: col*tile_w+w] = arr
    print(fname, arr.shape)
print(os.listdir(tile_folder))

# crop off any extra padding (edge tiles may be smaller)
crop_w = math.ceil(full_width  / level)
crop_h = math.ceil(full_height / level)
mosaic = mosaic[:crop_h, :crop_w]

# ────── cell 3: display ──────
plt.figure(figsize=(8,8))
plt.imshow(mosaic)
plt.axis('off')
plt.show()

IsADirectoryError: [Errno 21] Is a directory: '/Users/chris/Documents/lab/emAnalysis/backend/server/media/mims/mosaics/set_881924a6-1798-409b-8e0e-5405df9de9df/13C/mosaic_20250708T230647_13C.tif/14336,3584,512,512'